# Graph DataBase <br>
그래프 DB는 NoSql 의 한 종류로 다양한 OpenSource 가 존재하지만, 그중 가장 유명한 neo4j 를 가지고 개발중인 F/W 에 적용하고자 기본적인 기능들은 Jupyter 로 사전에 테스트 하고자 한다. 개발 환경은 Python, Django 기반으로 "Neo4j Python Driver"라는 Python 라이브러리를 사용하여 Server 형태로 구축된 Graph Db 에 접속하여 일반적인 Node 생성, Relation 생성, Match 를 사용한 조회 작업등을 테스트할 예정이다. <br>
Neo4j 의 설치는 https://neo4j.com/download/community-edition/ 를 참조하면 간단하게 설치가 가능하며, 별다른 Setting 을 하지 않을 경우 기본 포트는 (api :7687, web:7474)가 된다. 아래의 예제들은 Neo4j가 설치되어 있다는 가정하에 수행된다.<br> 
<br><b>[그림1. graph data base]</b><br>
<img src="../images/graphdb.png"><br>
[가이드(Docker)](http://neo4j.com/docs/operations-manual/current/installation/docker/)

In [202]:
from neo4j.v1 import GraphDatabase, basic_auth
import requests
import json, os

url = "{0}:{1}".format('localhost' , "7687")

driver = GraphDatabase.driver("bolt://" + url, auth=basic_auth("neo4j", "1234"))
session = driver.session()

# Record 생성 
아래와 같이 Simple 하게 Lable 이 Person 이고 Property 가 name 인 Node를 만들어 보자. <br>

In [203]:
result = session.run("CREATE (you:Person {name:'You'})"
                     "RETURN you")
for record in result : 
    print("you : {0}".format(record["you"]))


you : <Node id=134 labels={'Person'} properties={'name': 'You'}>


# Relation 생성 - 직업
Record 를 만들고 Relation 을 만들면 http://localhost:7474 에서 아래와 같은 그래프를 확인 할 수 있다.<br> 실행을 두번하여서 Person:you Node와 DataBase:Neo4j Node 사이에 관계가 Like 가 정의된 것을 확인 할 수 있다. <br> 
<img src="../images/graph_1.png"><br>

In [204]:
result = session.run("MATCH  (you:Person {name:'You'})"
                     "CREATE (you)-[like:LIKE]->(neo:Database {name:'Neo4j' })"
                     "RETURN you,like,neo")

for record in result : 
    print("{0}/{1}/{2}".format(record["you"], record["like"], record["neo"]))

<Node id=134 labels={'Person'} properties={'name': 'You'}>/<Relationship id=179 start=134 end=135 type='LIKE' properties={}>/<Node id=135 labels={'Database'} properties={'name': 'Neo4j'}>


# Relation 생성 - 친구
아래와 같이 (you)-[:FRIEND]->(:Person {name:name} 관계를 추가적으로 생성하여 보자. <br>
아래와 같이 친구들이 추가된 것을 볼 수 있다. <br>
<img src="../images/graph_2.png"><br>

In [205]:
session.run("MATCH (you:Person {name:'You'})"
            "FOREACH (name in ['Johan','Rajesh','Anna','Julia','Andrew'] |"
            "  CREATE (you)-[:FRIEND]->(:Person {name:name}))")


In [206]:
result = session.run("MATCH (you {name:'You'})-[:FRIEND]->(yourFriends)"
                     "RETURN you, yourFriends")
for record in result :
    print("you : {0} , friends : {1}".format(record['you']['name'],record['yourFriends']['name'] ))

you : You , friends : Julia
you : You , friends : Andrew
you : You , friends : Rajesh
you : You , friends : Anna
you : You , friends : Johan
you : You , friends : Andrew
you : You , friends : Julia
you : You , friends : Anna
you : You , friends : Rajesh
you : You , friends : Johan
you : You , friends : Andrew
you : You , friends : Anna
you : You , friends : Julia
you : You , friends : Johan
you : You , friends : Rajesh
you : You , friends : Julia
you : You , friends : Andrew
you : You , friends : Rajesh
you : You , friends : Anna
you : You , friends : Johan
you : You , friends : Andrew
you : You , friends : Rajesh
you : You , friends : Johan
you : You , friends : Julia
you : You , friends : Anna
you : You , friends : Andrew
you : You , friends : Julia
you : You , friends : Johan
you : You , friends : Anna
you : You , friends : Rajesh
you : You , friends : Johan
you : You , friends : Rajesh
you : You , friends : Anna
you : You , friends : Julia
you : You , friends : Andrew
you : You , f

# 친구의 친구 검색
위에서 만든 친구중에 Anna 라는 친구의 친구 Amanda 를 추가하고 Amanda 의 직업은 Neo4j 로 설정한다. <br>
그리고 당신의 친구중에 전문분야가 Neo4j 인 친구를 찾는 Path를 탐색한다. 

In [207]:
session.run("MATCH (neo:Database {name:'Neo4j'})"
            "MATCH (anna:Person {name:'Anna'})"
            "CREATE (anna)-[:FRIEND]->(:Person:Expert {name:'Amanda'})-[:WORKED_WITH]->(neo)")

In [208]:
result = session.run("MATCH (you {name:'You'})"
                     "MATCH (expert)-[:WORKED_WITH]->(db:Database {name:'Neo4j'})"
                     "MATCH path = shortestPath( (you)-[:FRIEND*..5]-(expert) )"
                     "RETURN db,expert,path")

for record in result :
    print("expert:{0}".format(record['expert'] ))
    print("path:{0}".format(record['path'] ))

expert:<Node id=141 labels={'Person', 'Expert'} properties={'name': 'Amanda'}>
path:<Path start=134 end=141 size=2>


# 구글의 Syntaxnet 사용해보자
 - Python 2.7에서만 구동되기에 Docker을 통해 Rest로 호출 (느림)
![](https://2.bp.blogspot.com/-fqtmVS97tOs/VzTEAI9BQ8I/AAAAAAAAA_U/xPj0Av64sGseS0rF4Z1BbhmS77J-HuEvwCLcB/s1600/image04.gif)
[Universal Dependency Relations](http://universaldependencies.org/u/dep/all.html)

In [209]:
#-*-coding: utf-8 -*-

import requests
import json, os
url = "{0}:{1}".format("IP" , "9000")

#https://github.com/TensorMSA/tensormsa_syntax_docker
resp = requests.post('http://' + 'localhost:9000' + '/api/v1/query',
                         json={
                                 #"strings": [["나는 학교에 지금 간다"]]
                                 "strings": [["You friends whose job is Neo4j"]]

                         }
                     )

print(json.dumps(resp.json(),ensure_ascii=False))


[{"dep": "ROOT", "name": "friends", "contains": [{"dep": "dep", "name": "You", "pos_tag": "PRP"}, {"dep": "rcmod", "name": "Neo4j", "contains": [{"dep": "nsubj", "name": "job", "contains": [{"dep": "poss", "name": "whose", "pos_tag": "WP$"}], "pos_tag": "NN"}, {"dep": "cop", "name": "is", "pos_tag": "VBZ"}], "pos_tag": "NNP"}], "pos_tag": "NNS"}]


In [210]:
print(json.dumps(resp.json()[0]['dep'],ensure_ascii=False))
print(json.dumps(resp.json()[0]['name'],ensure_ascii=False))
root = json.dumps(resp.json()[0]['name'])

"ROOT"
"friends"


In [211]:
print(json.dumps(resp.json()[0]['contains'][0]['name'],ensure_ascii=False))
contains = json.dumps(resp.json()[0]['contains'][0]['name'])

"You"


In [212]:
print(json.dumps(resp.json()[0]['contains'][1]['name'],ensure_ascii=False))
contains2 = json.dumps(resp.json()[0]['contains'][1]['name'])

"Neo4j"


In [213]:
result = session.run("MATCH (you {name:"+ contains +"})"
                     "MATCH (expert)-[:WORKED_WITH]->(db:Database {name:" + contains2 +"})"
                     "MATCH path = shortestPath( (you)-[:FRIEND*..5]-(expert) )"
                     "RETURN db,expert,path")

for record in result :
    print("expert:{0}".format(record['expert'] ))
    print("path:{0}".format(record['path'] ))

expert:<Node id=141 labels={'Person', 'Expert'} properties={'name': 'Amanda'}>
path:<Path start=134 end=141 size=2>


## Data 삭제

In [214]:
session.run("MATCH (n:Database) remove n:Database RETURN n")
session.run("MATCH (n:Expert) remove n:Expert RETURN n")
session.run("MATCH (n:Person) remove n:Person RETURN n")
session.run("MATCH (n:Friend) remove n:Friend RETURN n")


In [216]:
session.close()
print("close")

close
